In [3]:
# check the pytorch installation
import torch
print(torch.cuda.is_available())       # Should be True
print(torch.backends.cudnn.enabled)    # Should be True

True
True


In [5]:
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import csv
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
#import cv2

In [0]:
# camera-ready
import sys


sys.path.append("/model")
from deeplabv3 import DeepLabV3

sys.path.append("/utils")
from utils import label_img_to_color


In [0]:
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import csv
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

In [6]:
network = DeepLabV3("eval_val", project_dir="deeplabv3").cuda()

pretrained resnet, 18


In [7]:
network.load_state_dict(torch.load("model_13_2_2_2_epoch_580.pth"))

<All keys matched successfully>

In [0]:
class DatasetVal(torch.utils.data.Dataset):
    def __init__(self, data_path):
        self.img_dir = data_path
        self.img_h = 1024
        self.img_w = 2048

        self.examples = []
        sub_dirs = os.listdir(self.img_dir)
        for sub_dir in sub_dirs:
          file_names = os.listdir(self.img_dir + '/' + sub_dir)
          for file_name in file_names:
              if file_name.startswith('p1s'):
                img_id = file_name.split('.')[0]
                img_path = self.img_dir + sub_dir + '/' + file_name
                example = {}
                example["img_path"] = img_path
                example["img_id"] = img_id
                self.examples.append(example)

        self.num_examples = len(self.examples)
        
    def __getitem__(self, index):
        example = self.examples[index]

        img_id = example["img_id"]
        img_path = example["img_path"]
        img = cv2.imread(img_path, -1) # (shape: (1024, 2048, 3))

        # normalize the img (with the mean and std for the pretrained ResNet):
        img = img/255.0
        img = img - np.array([0.485, 0.456, 0.406])
        img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
        img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
        img = img.astype(np.float32)

        # convert numpy -> torch:
        img = torch.from_numpy(img) # (shape: (3, 512, 1024))
        #label_img = torch.from_numpy(label_img) # (shape: (512, 1024))
        print(img_id)
        return (img, img_id)

    def __len__(self):
        return self.num_examples

In [0]:
val_dataset = DatasetVal(data_path="Dataset/")

In [11]:
len(val_dataset)

0

In [0]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=1, shuffle=False,
                                         num_workers=1)

In [13]:
len(val_loader)

0

In [0]:
network.eval()

DeepLabV3(
  (resnet): ResNet_BasicBlock_OS8(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [0]:
#produce labeled image
for step, (imgs, img_ids) in enumerate(val_loader):
    #print(img_ids)
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))

  
        ########################################################################
        # save data for visualization:
        ########################################################################
        
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            if i == 0:
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)
                #producing original predicted images
                cv2.imwrite(network.model_dir + "/" + img_id + "_pred_label_img.png", pred_label_img )
                #producing colored predicted images
                #cv2.imwrite(network.model_dir + "/" + img_id + "_pred_label_img_color.png", pred_label_img_color )
                #producing overlayed images
                #cv2.imwrite(network.model_dir + "/" + img_id + "_overlayed.png", overlayed_img)
              

p1s00027
p1s00033


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


p1s00543


KeyboardInterrupt: ignored

In [0]:
num_classes = 20

In [0]:
dir_path = network.model_dir 
file_names = os.listdir(dir_path)

In [0]:
#save category ratios to csv file
with open('object_ratios.csv', 'w') as output:
  writer = csv.writer(output)
  writer.writerow(['id'] + [i for i in range(20)])
  for file in file_names:
    if file[-3:] =='png':
      trainId_to_count = {}
      for trainId in range(num_classes):
        trainId_to_count[trainId] = 0
      pred_label_img = cv2.imread(dir_path + '/' + file, -1)
      for trainId in range(0, num_classes):
          # count how many pixels in pred_label_img which are of object class trainId:
        trainId_mask = np.equal(pred_label_img, trainId)
        trainId_count = np.sum(trainId_mask)

          # add to the total count:
        trainId_to_count[trainId] += trainId_count
      total_count = sum(trainId_to_count.values())
      probs = []
      for trainId, count in trainId_to_count.items():
        trainId_prob = float(count)/float(total_count) 
        probs.append(trainId_prob)
      id = file.split('.')[0]
      writer.writerow([id] + probs)

KeyboardInterrupt: ignored